### Import Required Libraries and Set Up Environment Variables

In [283]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

# I'm adding this url parse because the field_query seems to be breaking the url and this may fix it
import urllib.parse

# I had to add this to help with the reorganization of the dataframe into a proper chart
import re

In [284]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [285]:
# Set the base URL

url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
encoded_filter_query = urllib.parse.quote(query)
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"
begin_date = "20130101"
end_date = "20230531"

sort = "newest"


query_url = f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&fq={encoded_filter_query}&sort={sort}"
query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=0Ynj9P6CZNRvyd9EWKoByLOXWX1o2Wp2&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&fq=section_name%3A%22Movies%22%20AND%20type_of_material%3A%22Review%22%20AND%20headline%3A%22love%22&sort=newest'

In [273]:

# Create an empty list to store the reviews
NYT_Reviews_List = []

# loop through pages 0-19
for page in range(20):
    
    # create query with a page number
    query_url = f"{query_url}&page{page}"
    
# API results show 10 articles at a time
    response = requests.get(query_url)
    data = response.json()
    
    # Make a "GET" request and retrieve the JSON

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        if 'response' in data and 'docs' in data['response']:
            for article in data['response']['docs']: 
                NYT_Reviews_List.append(article)
            print(f"Retrieved page {page}")
        else:
            print(f"No results found on page {page}")
            break
    except Keyerror as e:
        print(f"Error on page {page}: {str(e)}")
        break

        # loop through the reviews["response"]["docs"] and append each review to the list

        # Print the page that was just retrieved


        # Print the page number that had no results then break from the loop



Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19


In [274]:
response = requests.get(NYT_query_url)
print(response.status_code)
print(response.json())

200
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [], 'meta': {'hits': 0, 'offset': 0, 'time': 219}}}


In [275]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
NYT_dataframe = pd.json_normalize(NYT_Reviews_List)
NYT_dataframe

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None


In [276]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"

NYT_dataframe['title'] = df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

NYT_dataframe

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine


In [277]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keywords):
    if isinstance(keywords, list) and all(isinstance(d, dict) for d in keywords):
        return ', '.join([keyword['value'] for keyword in keywords])
    elif isinstance(keywords, str):
        return keywords
    return ''
    # Apply the extract_keywords function to the "keywords" column
NYT_dataframe['keywords'] = NYT_dataframe['keywords'].apply(extract_keywords)

# Extract the movie title from the "headline.main" column
NYT_dataframe['title'] = NYT_dataframe['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
NYT_dataframe.head(10)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"Movies, Kapur, Shekhar, James, Lily, Azmi, Sha...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"Movies, You Can Live Forever (Movie), Slutsky,...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"Movies, A Tourist's Guide to Love (Movie), Tsu...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"Movies, Zlotowski, Rebecca, Other People's Chi...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"Movies, Bracey, Luke (1989- ), Liu, Simu, Reid...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"Documentary Films and Programs, The Lost Weeke...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"Movies, A Thousand and One (Movie), Rockwell, ...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"Movies, Your Place or Mine (Movie), Witherspoo...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The

In [278]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

Selected_Movie_Titles = NYT_dataframe['title'].to_list()
print(Selected_Movie_Titles[:200])


['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Childr

### Access The Movie Database API

In [297]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [299]:
# Create an empty list to store the results

tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in Selected_Movie_Titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
       time.sleep(10)  # Sleep for 10 seconds every 50 requests
 

    # Add 1 to the request counter

    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    try:
        response = requests.get(url + title + tmdb_key_string)
        data = response.json()
        
        if data['results']:
            movie = data['results'][0]
            
            # Get movie id
            movie_id = movie['id']
            
            # Make a request for the full movie details
            movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            movie_response = requests.get(movie_url)
            movie_data = movie_response.json()
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in movie_data['genres']]
            
            # Extract the spoken_languages' English name into a list
            languages = [lang['english_name'] for lang in movie_data['spoken_languages']]
            
            # Extract the production_countries' name into a list
            countries = [country['name'] for country in movie_data['production_countries']]
            
            # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
            tmdb_movies_list.append({
                'title': movie['title'],
                'tmdb_id': movie_id,
                'genres': genres,
                'languages': languages,
                'countries': countries,
                'overview': movie['overview'],
                'release_date': movie['release_date'],
                'vote_average': movie['vote_average']
            })
            
            # Print out the title that was found
            print(f"Found: {movie['title']}")
        else:
            print(f"Movie not found: {title}")
    
    except Exception as e:
        print(f"Error processing {title}: {str(e)}")



Found: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People's Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People's Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People's Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
F

In [302]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview = tmdb_movies_list[:5]
formatted_preview = json.dumps(preview, indent=4)
print(formatted_preview)

[
    {
        "title": "The Attachment Diaries",
        "tmdb_id": 743040,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "languages": [
            "Spanish"
        ],
        "countries": [
            "Argentina"
        ],
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "release_date": "2021-10-07",
        "vote_average": 3.0
    },
    {
        "title": "You Can Live Forever",
        "tmdb_id": 887580,
        "genres": [
            "Drama",
            "Romance"
        ],
        "languages": [
            "English",
         

In [308]:
# Convert the results to a DataFrame
TMDB_Dataframe = pd.DataFrame(tmdb_movies_list)
TMDB_Dataframe.head(10)

,title,tmdb_id,genres,languages,countries,overview,release_date,vote_average
0,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.000
1,You Can Live Forever,887580,"[Drama, Romance]","[English, French]","[Canada, United States of America]","When Jaime, a gay teenager, is sent to live in...",2023-03-24,6.474
2,A Tourist's Guide to Love,813726,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],"After an unexpected break up, a travel executi...",2023-04-21,6.290
3,Other People's Children,803694,"[Drama, Comedy]","[French, English]",[France],"Rachel loves her life, her students, her frien...",2022-09-21,6.818
4,One True Loves,846961,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]",Emma and Jesse are living the perfect life tog...,2023-04-07,6.520
5,The Lost Weekend: A Love Story,965042,[Documentary],[English],[United States of America],May Pang lovingly recounts her life in rock & ...,2023-04-13,6.000
6,A Thousand and One,855263,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America],Struggling but unapologetically living on her ...,2023-03-31,6.871
7,Your Place or Mine,703451,"[Romance, Comedy]",[English],[United States of America],When best friends and total opposites Debbie a...,2023-02-10,6.280
8,Love in the Time of Fentanyl,976653,[Documentary],[English],"[Canada, United States of America]",An intimate portrait of a community fighting t...,2023-02-03,0.000
9,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.000


### Merge and Clean the Data for Export

In [320]:
# Merge the New York Times reviews and TMDB DataFrames on title
Merged_Dataframe = pd.merge(NYT_dataframe, TMDB_Dataframe, on='title', how='inner')
Merged_Dataframe.head(10)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.person,byline.organization,title,tmdb_id,genres,languages,countries,overview,release_date,vote_average
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
5,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
6,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
7,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachm

In [332]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'languages', 'countries']

# Create a list of characters to remove
chars_to_remove = ['[', ']', "'"]

# Create a copy of the original dataframe to modify
Merged_Dataframe_fixed = Merged_Dataframe.copy()

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    Merged_Dataframe_fixed[column] = Merged_Dataframe_fixed[column].astype(str)
    
    # Loop through characters to remove
    for char in chars_to_remove:
        Merged_Dataframe_fixed[column] = Merged_Dataframe_fixed[column].str.replace(char, '')
    
    # Remove any leading/trailing whitespace
    Merged_Dataframe_fixed[column] = Merged_Dataframe_fixed[column].str.strip()

# Display the fixed DataFrame in the nice chart format
Merged_Dataframe_fixed.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.person,byline.organization,title,tmdb_id,genres,languages,countries,overview,release_date,vote_average
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0


In [334]:
# Drop "byline.person" column
Merged_Dataframe_fixed.drop('byline.person', axis=1, inplace=True)
Merged_Dataframe_fixed.head(10)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.original,byline.organization,title,tmdb_id,genres,languages,countries,overview,release_date,vote_average
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
5,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
6,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
7,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.0
8,https://www.nytimes.com/2023/05/25/movies/the-.

In [350]:
list_columns = [col for col in Merged_Dataframe.columns if Merged_Dataframe[col].apply(lambda x: isinstance(x, list)).any()]

# Convert list columns to strings
for col in list_columns:
    Merged_Dataframe[col] = Merged_Dataframe[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

# Now remove duplicate rows
Merged_Dataframe = Merged_Dataframe.drop_duplicates()

# Reset the index
Merged_Dataframe = Merged_Dataframe.reset_index(drop=True)

# Display the DataFrame to verify
display(Merged_Dataframe.head())

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.person,byline.organization,title,tmdb_id,genres,languages,countries,overview,release_date,vote_average
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"{'firstname': 'Jeannette', 'middlename': None,...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"Argentina, 1970s. A desperate young woman goes...",2021-10-07,3.000
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"Movies, You Can Live Forever (Movie), Slutsky,...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"{'firstname': 'Elisabeth', 'middlename': None,...",None,You Can Live Forever,887580,"Drama, Romance","English, French","Canada, United States of America","When Jaime, a gay teenager, is sent to live in...",2023-03-24,6.474
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"Movies, Bracey, Luke (1989- ), Liu, Simu, Reid...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,"{'firstname': 'Brandon', 'middlename': None, '...",None,One True Loves,846961,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",Emma and Jesse are living the perfect life tog...,2023-04-07,6.520
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"Documentary Films and Programs, The Lost Weeke...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,"{'firstname': 'Glenn', 'middlename': None, 'la...",None,The Lost Weekend: A Love Story,965042,Documentary,English,United States of America,May Pang lovingly recounts her life in rock & ...,2023-04-13,6.000
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"Movies, A Thousand and One (Movie), Rockwell, ...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,"{'firstname': 'Manohla', 'middlename': None, '...",None,A Thousand and One,855263,"Drama, Crime","English, Spanish, Portuguese",United States of America,Struggling but unapologetically living on her ...,2023-03-31,6.871


In [352]:
# Export data to CSV without the index

Merged_Dataframe.to_csv('collected_data.csv', index=False)